## From Principal Component (PCA) to Direct Coupling Analysis (DCA) of Coevolution in Proteins

This notebook takes a look at a 2013 paper from Simona Cocco, Remi Monasson, Martin Weigt titled
**From Principal Component to Direct Coupling Analysis of Coevolution in Proteins: Low-Eigenvalue Modes are Needed for Structure Prediction.** *\[2013Cocco\]*

Link: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003176

This paper looks at extracting functional and structural information from Multiple Sequence Alignments (MSA) of homologous proteins. First a covariance matrix of the residues are created from the MSA. Then the paper connects two approaches 

*  PCA - which identifies correlated groups of residues
*  DCA - which identifies residue-residue contacts

It shows how these two methods are related in non-intuitive ways using sophisticated statistical-physics models. This connection between the two approaches allows one to perform some sort of "dimension reduction" on DCA and to accurately predict residue-residue contacts with a smaller number of parameters. It also shows that the low eigenvalue values, which are discarded by PCA, are actually important to recover contact information. 

### Sections

1. [Multiple Sequence Alignment](#msa)
2. [Re-weighting Sequences](#reweight)
3. [Compute Single and Double Site marginals](#marginals)
4. [Compute the Covariance matrix](#covmat)

***


### Multiple Sequence Alignment (MSA)  <a id="msa"></a>

We use a [DHFR](https://www.uniprot.org/uniprot/P00374) alignment as an example to try out the methods of the papers. The alignment file is generated by [jackhmmer](https://www.ebi.ac.uk/Tools/hmmer/search/jackhmmer)


```
sp|P00374|DYR_HUMAN Dihydrofolate reductase OS=Homo sapiens OX=9606 GN=DHFR PE=1 SV=2
MVGSLNCIVAVSQNMGIGKNGDLPWPPLRNEFRYFQRMTTTSSVEGKQNLVIMGKKTWFS
IPEKNRPLKGRINLVLSRELKEPPQGAHFLSRSLDDALKLTEQPELANKVDMVWIVGGSS
VYKEAMNHPGHLKLFVTRIMQDFESDTFFPEIDLEKYKLLPEYPGVLSDVQEEKGIKYKF
EVYEKND
```

Note: Not exactly sure how the alignment is generated. 

```shell
jackhmmer -A DHFR_uniref90.aln --noali --notextw P00374.fasta uniref90.fasta
# some filtering script
```

In [1]:
import os
import itertools
import numpy as np

In [2]:
datadir = "../data"
msa_file = os.path.join(datadir, "DHFR.aln")

# Read all the lines in the file into a 2D array of type S1
with open(msa_file) as fh:
    arr = np.array([[x for x in line.strip()] for line in fh], np.dtype("S1"))

print("shape =", arr.shape, ",dtype= ", arr.dtype)

shape = (56165, 186) ,dtype=  |S1


In [3]:
# M is the number of sequences
# L is the length
M, L = arr.shape
print("Number of sequences : {} ".format(M))
print("Sequence Length : {}".format(L))

Number of sequences : 56165 
Sequence Length : 186


In [4]:
# the first sequence
arr[0, :].tostring()

b'VRPLNCIVAVSQNMGIGKNGDLPWPPLRNEFKYFQRMTTTSSVEGKQNLVIMGRKTWFSIPEKNRPLKDRINIVLSRELKEPPRGAHFLAKSLDDALRLIEQPELASKVDMVWIVGGSSVYQEAMNQPGHLRLFVTRIMQEFESDTFFPEIDLGKYKLLPEYPGVLSEVQEEKGIKYKFEVYEKKD'

In [5]:
# the second sequence
arr[1, :].tostring()

b'----SIVVVMCKRFGIGRNGVLPWSPLQADMQRFRSITAG-------GGVIMGRTTFDSIPEEHRPLQGRLNVVLTTSADLMKNSNIIFVSSFDELDAIVGL----HDHLPWHVIGGVSVYQHFLEKSQVTSMYVTFVDGSLECDTFFPHQFLSHFEITRA---SALMSDTTSGMSYRFVDYTR--'

We can order the amino acids any way we like. Here is a sorting based on some amino acid properties. 
https://proteinstructures.com/Structure/Structure/amino-acids.html

In [6]:
AMINO_ACIDS = np.array([aa for aa in "RKDEQNHSTCYWAILMFVPG-"], "S1")

### Compute the weights of each sequence <a id="reweight"></a>

To compute the weight of a sequence, we first compute the hamming distance between this sequence and all the other sequences in the alignment. Then we count the number of these distances that are less than a cutoff. 

This count is 1 for isolated sequences. It is large for sequences that have many similar sequences in the alignment. The weight is reciprocal of the count. So it is 1 for isolated sequences and close to zero for sequences that have many similar sequences in the MSA. (Eqn 27 in 2013Cocco)

$$w_m = \frac{1}{ \| \{ n | 1 \leq n \leq M  ; d_H [ (a_1^n, \ldots, a_L^n), (a_1^m, \ldots, a_L^m) ] \leq xL \} \| }$$



In [7]:
hamming_cutoff = 0.2 # This is x in the equation above
def compute_weight(index, x=hamming_cutoff, arr=arr):
    hamming_distances = np.sum(arr[index, :] != arr, axis=1)
    count = np.sum(hamming_distances <= x * arr.shape[1]) # L = arr.shape[1]
    return (1.0 / count)

# compute the weight of the first sequence
compute_weight(0)

0.005681818181818182

In [8]:
progress_bar = True
try:
    from IPython.display import clear_output
except ImportError:
     progress_bar = False

weights_file = os.path.join(datadir, "DHFR.weights.npy")

if os.path.isfile(weights_file):
    weights = np.load(weights_file)
    print("Loading weights from : ", weights_file)

else:
    weights = np.zeros(M)

    for i in range(M):
        weights[i] = compute_weight(i)
        if i % 100 == 0:
            if progress_bar:
                clear_output(wait=True)
            print ("Processing sequence", i, "of", M)
    np.save(weights_file, weights)
    print("Finished computing sequence weights and saved to : ", weights_file)


Loading weights from :  ../data/DHFR.weights.npy


In [9]:
# number of effective sequences
M_eff = sum(weights) # Eqn 28 in 2013Cocco
print(int(round(M_eff)))

15238


In [10]:
# q is the alphabet
q = len(AMINO_ACIDS)
pseudo_count = round(M_eff)

### Compute Weighted Single and Double site marginals <a id="marginals"></a>

We first compute the weighted counts $$\sum_{m=1}^M w_m \delta_{a, a_i^m}$$ and then $$\sum_{m=1}^M w_m \delta_{a, a_i^m} \delta_{b, a_j^m}$$ and save them to disk. The second computation takes a long time since the arrays are too large to compute outer products and broadcast them. 

To get the marignals we need to divide these counts above by the sum of the weights. However, we only do this after adding a pseudocount in Eqns 29 and 30 in 2013Cocco.


In [11]:
single_site_marginal_file = os.path.join(datadir, "DHFR.single.npy")
double_site_marginal_file = os.path.join(datadir, "DHFR.double.npy")

if os.path.isfile(double_site_marginal_file) and os.path.isfile(single_site_marginal_file):
    f_i_a = np.load(single_site_marginal_file)
    print("Loading single site marginals from ", single_site_marginal_file)

    f_i_j_a_b = np.load(double_site_marginal_file)
    print("Loading double site marginals from ", double_site_marginal_file)    

else:
    # We first compute a one-hot matrix of shape (M, L, q)
    # which is 0/1 in index (m, i, a) depending on whether 
    # Protein *m* has amino acid *a* in position *i* 
    arr_onehot = np.zeros(arr.shape + (q,), dtype=np.uint8)
    for i, a in enumerate(AMINO_ACIDS):
        arr_onehot[..., i] = (arr == a)
    print("arr_onehot.shape = {}".format(arr_onehot.shape))
    
    # we reorder the one-hot axes so that the sequences are in the last dimension
    # this allows us to multiply easily by the weights using broadcasting
    arr_onehot_reorder = np.moveaxis(arr_onehot, 0, 2)
    weighted_arr_onehot = arr_onehot_reorder * weights

    # Weighted Single Site Marignals
    f_i_a = np.sum((arr_onehot_reorder * weights), axis=-1)
    print("f_i_a.shape = {}".format(f_i_a.shape)) 
    
    # Set up the weighted double site marginals array
    f_i_j_a_b = np.zeros((L, q, L, q), dtype=f_i_a.dtype)
    
    # we cannot use outer products here because our arrays are too big
    # So we iterate
    for j, b in itertools.product(range(L), range(q)):
        f_i_j_a_b[:, :, j, b] = np.sum((weighted_arr_onehot * arr_onehot_reorder[j, b, :]), axis=-1)
    if progress_bar:
        clear_output(wait=True)
    print("Finished processing j={}, b={}, AA={}".format(j, b, AMINO_ACIDS[b].tostring().decode()))

    np.save(single_site_marginal_file, f_i_a)
    np.save(double_site_marginal_file, f_i_j_a_b)
    
    # delete large temporary arrays
    del weighted_arr_onehot, arr_onehot_reorder, arr_onehot
    print("Finished computing single and double site marginals and saved to cache files")

Loading single site marginals from  ../data/DHFR.single.npy
Loading double site marginals from  ../data/DHFR.double.npy


In [12]:
# Add Pseudo count and compute the marginals (Eqn 29 and 30 2013Cocco)

f_i_a = 1. / (M_eff + pseudo_count) * (pseudo_count/q + f_i_a)
f_i_j_a_b = 1. / (M_eff + pseudo_count) * (pseudo_count/(q*q) + f_i_j_a_b)

### Compute the covariance matrix <a id="covmat"></a>



In [13]:
# Covariance Matrix
# We take an outer product of f_i_a with itself using numpy's broadcasting rules. 
# This gives us a matrix where the (i,a, j, b) index is f[i,a] * f[j,b]
C_i_j_a_b = f_i_j_a_b  - f_i_a[:, :, np.newaxis, np.newaxis] * f_i_a[np.newaxis, np.newaxis, :, :] 

# we project the covariance matrix down the first q-1 elements
C_i_j_a_b = C_i_j_a_b[:, :(q-1), :, :(q-1)]
print("C_i_j_a_b.shape = {}".format(C_i_j_a_b.shape)) 


C_i_j_a_b.shape = (186, 20, 186, 20)
